# Implementación Decision Trees - Clasificación
**Autor: Ronald Borja Román**

In [3]:
# Librerías
import numpy as np 
import pandas as pd 

In [89]:
import numpy as np

# Clase Nodo: 
class Nodo():
    # Constructor -> 
    def __init__(self, indice_atributo=None, umbral=None, izquierdo=None, derecho=None, valor=None):
        # Nodos de decisión 
        self.indice_atributo = indice_atributo
        self.umbral = umbral
        self.izquierdo = izquierdo
        self.derecho = derecho
        
        # Nodos finales (hojas)
        self.valor = valor

    
class ArbolDecisionClassifier():
    # Constructor del modelo
    def __init__(self, min_muestras_division=2, max_profundidad=2):
        self.min_muestras_division = min_muestras_division
        self.max_profundidad = max_profundidad
        self.raiz = None

    # Función para construir los nodos de forma recursiva 
    def construir_arbol(self, conjunto_datos, profundidad_actual=0):
        X, Y = conjunto_datos[:, :-1], conjunto_datos[:, -1]
        num_muestras, num_atributos = np.shape(X)

        # Condiciones de parada -> 
        if num_muestras < self.min_muestras_division or profundidad_actual == self.max_profundidad:
            valor_hoja = np.bincount(Y.astype(int)).argmax()
            return Nodo(valor=valor_hoja)

        # Mejor separador de los datos 
        mejor_separacion = self.obtener_mejor_separacion(conjunto_datos, num_muestras, num_atributos)

        # Calculo de 
        if mejor_separacion["info_gain"] > 0:
            izquierdo = self.construir_arbol(mejor_separacion["conjunto_izquierdo"], profundidad_actual + 1)
            derecho = self.construir_arbol(mejor_separacion["conjunto_derecho"], profundidad_actual + 1)
            
            return Nodo(indice_atributo=mejor_separacion["indice_atributo"],
                        umbral=mejor_separacion["umbral"], izquierdo=izquierdo, derecho=derecho)

        # Si no se cumple la conidición de la ganancia -> Se asume que es un nodo hoja 
        valor_hoja = np.bincount(Y).argmax()
        return Nodo(valor=valor_hoja)

    # Función para separar los datos 
    def obtener_mejor_separacion(self, conjunto_datos, num_muestras, num_atributos):
        mejor_separacion = {"info_gain": -float("inf")}
        
        # Se itera sobre el rango de todos los inputs al modelo
        for ind_atributo in range(num_atributos):
            valores_atributo = conjunto_datos[:, ind_atributo]
            umbrales_posibles = np.unique(valores_atributo) #Se toman solo los valores unicos 

            # Se evaluan todos los umbrales posibles -> 
            for umbral in umbrales_posibles:
                # División del nodo
                conjunto_izquierdo, conjunto_derecho = self.dividir(conjunto_datos, ind_atributo, umbral)

                # Siempre que los conjuntos tengan datos: 
                if len(conjunto_izquierdo) > 0 and len(conjunto_derecho) > 0:
                    info_gain_actual = self.info_gain(conjunto_datos[:, -1], conjunto_izquierdo[:, -1],
                                                      conjunto_derecho[:, -1])
                    
                    # Si la ganancia de información es mayor que la actual, se procede a almacenar el nuevo split 
                    if info_gain_actual > mejor_separacion["info_gain"]:
                        mejor_separacion = {"indice_atributo": ind_atributo,
                                            "umbral": umbral,
                                            "conjunto_izquierdo": conjunto_izquierdo,
                                            "conjunto_derecho": conjunto_derecho,
                                            "info_gain": info_gain_actual}

        return mejor_separacion
    
    # Función para dividir los datos 
    def dividir(self, conjunto_datos, ind_atributo, umbral):
        conjunto_izquierdo = conjunto_datos[conjunto_datos[:, ind_atributo] <= umbral]
        conjunto_derecho = conjunto_datos[conjunto_datos[:, ind_atributo] > umbral]
        return conjunto_izquierdo, conjunto_derecho

    # Función para calcular la ganancia usando la entrpía 
    def info_gain(self, padre, izquierdo, derecho):
        peso_izquierdo = len(izquierdo) / len(padre)
        peso_derecho = len(derecho) / len(padre)

        return self.entropy(padre) - (peso_izquierdo * self.entropy(izquierdo) + peso_derecho * self.entropy(derecho))

    # Función entropía 
    def entropy(self, y):
        prob_clases = np.bincount(y.astype(int)) / len(y)
        return -np.sum(prob_clases * np.log2(prob_clases + 1e-10))

    # Función para entrenar el modelo 
    def fit(self, X, Y):
        conjunto_datos = np.column_stack((X, Y))
        self.raiz = self.construir_arbol(conjunto_datos)
        
    # Función predict
    def predict(self, X):
        # Inicializar un array para almacenar las predicciones
        predicciones = np.zeros(len(X), dtype=int)

        # Iterar sobre cada muestra y predecir
        for i in range(len(X)):
            predicciones[i] = self.predict_single(self.raiz, X[i])
            
        return predicciones

    # Función auxiliar para predecir una sola muestra
    def predict_single(self, nodo, muestra):
        # Si es un nodo hoja, retornar el valor
        if nodo.valor is not None:
            return nodo.valor

        # Comparar el valor del atributo en la muestra con el umbral del nodo actual
        if muestra[nodo.indice_atributo] <= nodo.umbral:
            # Recurrir al subárbol izquierdo
            return self.predict_single(nodo.izquierdo, muestra)
        else:
            # Recurrir al subárbol derecho
            return self.predict_single(nodo.derecho, muestra)

In [55]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv('Iris.csv', skiprows=1, header=None, names=col_names)

data["type"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [107]:
data

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


In [57]:
X = data.iloc[:, :-1].values
y = data.iloc[:,-1].values.reshape(-1,1)

In [58]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_numeric = label_encoder.fit_transform(y.ravel())

In [59]:
labels_numeric

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [63]:
y = labels_numeric
y = y.astype('int64')

In [90]:
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
clf = ArbolDecisionClassifier()
clf.fit(X_train, Y_train)

In [100]:
preds = clf.predict(X_test) 

In [106]:
from sklearn.metrics import accuracy_score
print(f"La exactitud del modelo es de: {round(accuracy_score(Y_test, preds)*100,2)}%")

La exactitud del modelo es de: 96.67%
